In [4]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:

df_3m = pd.read_csv('datasets/btc_3m.csv')
df_3m.head()
df_3m.shape

(713017, 6)

In [48]:

df_5m = pd.read_csv('/content/drive/MyDrive/btc_5m.csv')
df_5m.head()
df_5m.shape

(427814, 6)

In [5]:


df_15m = pd.read_csv('/content/drive/MyDrive/btc_15m.csv')
df_15m.head()
df_15m.shape

(142610, 6)

In [6]:

df_30m = pd.read_csv('/content/drive/MyDrive/btc_30m.csv')
df_30m.head()
df_30m.shape

(71311, 6)

In [7]:

df_30m = pd.read_csv('/content/drive/MyDrive/btc_30m.csv')
df_30m.head()
df_30m.shape

(71311, 6)

In [8]:

import pandas as pd
df_1h = pd.read_csv('/content/drive/MyDrive/btc_1h.csv')
df_1h.head()
df_1h.shape

(35664, 6)

In [9]:

df_2h = pd.read_csv('/content/drive/MyDrive/btc_2h.csv')
df_2h.head()
df_2h.shape

(17842, 6)

In [10]:

df_4h = pd.read_csv('/content/drive/MyDrive/btc_4h.csv')
df_4h.head()
df_4h.shape

(8931, 6)

In [11]:
df_6h = pd.read_csv('/content/drive/MyDrive/btc_6h.csv')
df_6h.head()
df_6h.shape

(5959, 6)

In [7]:
def add_indicators(df):
    # SMA
    df['SMA_14'] = df['close'].rolling(window=14).mean()
    df['SMA_20'] = df['close'].rolling(window=20).mean()

    # RSI
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()

    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # Moving Average Convergence Divergence (MACD)
    short_window = 12
    long_window = 26
    signal_window = 9

    exp_short = df['close'].ewm(span=short_window, adjust=False).mean()
    exp_long = df['close'].ewm(span=long_window, adjust=False).mean()

    df['MACD'] = exp_short - exp_long
    df['Signal_Line'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()

    # Bollinger Bands
    window = 20
    df['Upper_Band'] = df['SMA_20'] + 2 * df['close'].rolling(window=window).std()
    df['Lower_Band'] = df['SMA_20'] - 2 * df['close'].rolling(window=window).std()

    # ATR
    # Calculate True Range (TR)
    df['High-Low'] = df['high'] - df['low']
    df['High-PrevClose'] = abs(df['high'] - df['close'].shift(1))
    df['Low-PrevClose'] = abs(df['low'] - df['close'].shift(1))

    df['TrueRange'] = df[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)

    # Choose the ATR period (e.g., 14 days)
    atr_period = 14
    df['EMA'] = df['close'].ewm(span=15, adjust=False).mean()
    # Calculate ATR using rolling mean
    df['ATR'] = df['TrueRange'].rolling(window=atr_period).mean()

    short_window = 20  # Example: 20-minute moving average
    long_window = 50   # Example: 50-minute moving average
    df['short_ma'] = df['close'].rolling(window=short_window, min_periods=1).mean()

    # Calculate long-term moving average (long_ma)
    df['long_ma'] = df['close'].rolling(window=long_window, min_periods=1).mean()

    # Drop intermediate columns used for calculation
    df.drop(['High-Low', 'High-PrevClose', 'Low-PrevClose', 'TrueRange'], axis=1, inplace=True)

    # Removing null rows
    df = df.dropna()
    return df

In [8]:
df=df_3m
add_indicators(df)


,datetime,open,high,low,close,volume,SMA_14,SMA_20,RSI,MACD,Signal_Line,Upper_Band,Lower_Band,EMA,ATR,short_ma,long_ma
19,2018-01-01 06:27:00,13528.98,13574.05,13452.56,13529.01,30.930862,13508.042143,13522.3515,63.328340,-27.865083,-35.726820,13632.476622,13412.226378,13530.660898,75.637143,13522.3515,13522.351500
20,2018-01-01 06:30:00,13528.99,13571.74,13481.82,13530.98,9.865947,13514.540000,13514.9005,63.547282,-25.395626,-33.660581,13596.614868,13433.186132,13530.700786,77.830000,13514.9005,13522.762381
21,2018-01-01 06:33:00,13530.98,13530.98,13481.82,13510.85,17.731324,13519.600000,13512.0430,59.952793,-24.777270,-31.883919,13589.842233,13434.243767,13528.219438,77.872857,13512.0430,13522.220909
22,2018-01-01 06:36:00,13520.76,13520.76,13453.00,13453.00,9.125183,13516.259286,13505.9430,43.393508,-28.625255,-31.232186,13582.071306,13429.814694,13518.817008,76.426429,13505.9430,13519.211304
23,2018-01-01 06:39:00,13460.00,13479.00,13402.28,13469.99,24.721129,13512.607143,13503.6795,42.687566,-29.958517,-30.977452,13581.318419,13426.040581,13512.713632,76.316429,13503.6795,13517.160417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713012,2022-01-31 05:18:00,37919.05,37919.05,37876.94,37891.30,30.717900,38030.132857,37982.7615,34.163891,41.458357,69.054866,38246.277549,37719.245451,37960.053526,90.932857,37982.7615,37802.079400
713013,2022-01-31 05:21:00,37891.30,37946.56,37870.00,37918.80,28.369420,38013.627857,37992.4030,31.685240,34.768530,62.197599,38229.105519,37755.700481,37954.896835,90.589286,37992.4030,37809.655600
713014,2022-01-31 05:24:00,37918.79,37948.79,37888.88,37898.01,22.346010,37993.218571,37998.0600,26.873331,27.472529,55.252585,38218.744796,37777.375204,37947.785981,88.400000,37998.0600,37817.149600
713015,2022-01-31 05:27:00,37898.02,37945.25,37870.00,37881.76,42.838730,37976.773571,38002.8590,29.526012,20.146915,48.231451,38207.713626,37798.004374,37939.532733,82.214286,38002.8590,37824.297600


In [84]:
df.iloc[49]

datetime       2018-01-01 09:35:00
open                      13447.73
high                      13455.05
low                        13370.0
close                      13400.0
volume                   20.443116
SMA_14                13451.181429
SMA_20                  13408.2405
RSI                      55.229492
MACD                     10.114115
Signal_Line              10.256253
Upper_Band             13601.78282
Lower_Band             13214.69818
ATR                         89.285
short_ma                13408.2405
long_ma                 13432.2132
EMA                   13432.629326
Name: 49, dtype: object

In [85]:
df.isna().sum()


datetime        0
open            0
high            0
low             0
close           0
volume          0
SMA_14         13
SMA_20         19
RSI            18
MACD            0
Signal_Line     0
Upper_Band     19
Lower_Band     19
ATR            13
short_ma        0
long_ma         0
EMA             0
dtype: int64

In [86]:
df1=df.dropna()


In [87]:
df1.iloc[0]

datetime       2018-01-01 07:05:00
open                      13500.16
high                       13522.1
low                        13477.2
close                     13515.89
volume                   26.855798
SMA_14                13510.225714
SMA_20                  13512.2735
RSI                      49.450965
MACD                    -13.622104
Signal_Line             -18.676401
Upper_Band            13607.882169
Lower_Band            13416.664831
ATR                      87.738571
short_ma                13512.2735
long_ma                 13512.2735
EMA                    13519.70014
Name: 19, dtype: object

In [9]:
def preprocessor(dataf):

  start_date = '2020-01-01'
  end_date = '2020-05-30'

  # Filter the data
  dataf1 = dataf[(dataf['datetime'] >= start_date) & (dataf['datetime'] <= end_date)]

  dataf1 = dataf1.fillna(0)

  num_timestamps = 10

  # Create new columns for each day
  for i in range(1, num_timestamps + 1):
      dataf1[f'openday{i}'] = dataf1['open'].shift(i)
      dataf1[f'highday{i}'] = dataf1['high'].shift(i)
      dataf1[f'lowday{i}'] = dataf1['low'].shift(i)
      dataf1[f'closeday{i}'] = dataf1['close'].shift(i)
      dataf1[f'volumeday{i}'] = dataf1['volume'].shift(i)
      dataf1[f'rsiday{i}'] = dataf1['rsi'].shift(i)
      dataf1[f'EMAday{i}'] = dataf1['EMA'].shift(i)

  # Drop rows with NaN values created by shifting
  dataf1 = dataf1.dropna()

  #drop the high low volume and rsi columns

  dataf1 = dataf1.drop(['high', 'low', 'volume'], axis=1)

  return dataf1


In [ ]:
dataf_3m=preprocessor(df_3m)

dataf_5m=preprocessor(df_5m)

dataf_15m=preprocessor(df_15m)

dataf_30m=preprocessor(df_30m)

dataf_1h=preprocessor(df_1h)

dataf_2h=preprocessor(df_2h)

dataf_4h=preprocessor(df_4h)

dataf_6h=preprocessor(df_6h)


,datetime,open,close,rsi,EMA,openday1,highday1,lowday1,closeday1,volumeday1,...,volumeday9,rsiday9,EMAday9,openday10,highday10,lowday10,closeday10,volumeday10,rsiday10,EMAday10
348390,2020-01-01 00:30:00,7169.32,7163.19,57.210464,7164.231590,7164.06,7170.59,7163.23,7168.86,15.076585,...,24.499807,49.891899,7162.907841,7168.22,7169.00,7157.80,7158.62,46.343030,53.100821,7162.853247
348391,2020-01-01 00:33:00,7161.77,7160.15,56.600338,7163.721391,7169.32,7170.41,7160.02,7163.19,180.417072,...,28.880407,44.946843,7162.508111,7158.05,7165.77,7158.00,7163.29,24.499807,49.891899,7162.907841
348392,2020-01-01 00:36:00,7160.15,7162.02,50.430778,7163.508718,7161.77,7163.76,7160.00,7160.15,34.999319,...,30.160449,49.033559,7163.005847,7163.29,7166.29,7157.75,7159.71,28.880407,44.946843,7162.508111
348393,2020-01-01 00:39:00,7162.96,7161.21,44.268710,7163.221378,7160.15,7163.27,7160.15,7162.02,18.167497,...,39.023847,57.995166,7163.971366,7158.35,7167.00,7158.35,7166.49,30.160449,49.033559,7163.005847
348394,2020-01-01 00:42:00,7160.87,7159.63,50.947645,7162.772456,7162.96,7166.32,7160.00,7161.21,23.535791,...,22.962410,63.001098,7164.853696,7165.14,7174.04,7165.14,7170.73,39.023847,57.995166,7163.971366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420145,2020-05-29 23:45:00,9412.90,9408.93,37.512656,9423.945880,9420.44,9421.50,9402.31,9412.90,131.110735,...,95.364586,53.592847,9432.598238,9433.86,9438.06,9427.22,9429.40,110.028078,55.657429,9432.030843
420146,2020-05-29 23:48:00,9408.93,9425.27,46.153846,9424.111395,9412.90,9416.13,9406.00,9408.93,62.352291,...,65.842809,47.400468,9432.799708,9429.39,9440.00,9424.89,9436.57,95.364586,53.592847,9432.598238
420147,2020-05-29 23:51:00,9425.25,9428.03,51.194539,9424.601221,9408.93,9427.04,9408.31,9425.27,73.968007,...,90.280471,40.415519,9432.102245,9436.43,9442.33,9433.42,9434.21,65.842809,47.400468,9432.799708
420148,2020-05-29 23:54:00,9428.04,9417.25,38.231966,9423.682318,9425.25,9434.13,9421.29,9428.03,120.602769,...,69.966933,44.927906,9431.470714,9433.79,9439.34,9424.13,9427.22,90.280471,40.415519,9432.102245


In [15]:

#features = ['open', 'high', 'low', 'volume','openday1',open,high,low,close,volume,rsi,openday1,highday1,lowday1,closeday1,openday2,highday2,lowday2,closeday2,openday3,highday3,lowday3,closeday3,openday4,highday4,lowday4,closeday4,openday5,highday5,lowday5,closeday5,openday6,highday6,lowday6,closeday6,openday7,highday7,lowday7,closeday7,openday8,highday8,lowday8,closeday8,openday9,highday9,lowday9,closeday9,openday10,highday10,lowday10,closeday10,volumeday1,rsiday1,volumeday2,rsiday2,volumeday3,rsiday3,volumeday4,rsiday4,volumeday5,rsiday5,volumeday6,rsiday6,volumeday7,rsiday7,volumeday8,rsiday8,volumeday9,rsiday9,volumeday10,rsiday10]
features= ['open','EMA','rsi','openday1', 'highday1', 'lowday1', 'closeday1', 'volumeday1', 'rsiday1',
       'EMAday1', 'openday2', 'highday2', 'lowday2', 'closeday2', 'volumeday2',
       'rsiday2', 'EMAday2', 'openday3', 'highday3', 'lowday3', 'closeday3',
       'volumeday3', 'rsiday3', 'EMAday3', 'openday4', 'highday4', 'lowday4',
       'closeday4', 'volumeday4', 'rsiday4', 'EMAday4', 'openday5', 'highday5',
       'lowday5', 'closeday5', 'volumeday5', 'rsiday5', 'EMAday5']
target = 'close'


In [ ]:
x_3m = dataf_3m[features]
y_3m = dataf_3m[target]

x_5m = dataf_5m[features]
y_5m = dataf_5m[target]

x_15m = dataf_15m[features]
y_15m = dataf_15m[target]

x_30m = dataf_30m[features]
y_30m = dataf_30m[target]

x_1h = dataf_1h[features]
y_1h = dataf_1h[target]

x_2h = dataf_2h[features]
y_2h = dataf_2h[target]

x_4h = dataf_4h[features]
y_4h = dataf_4h[target]

x_6h = dataf_6h[features]
y_6h = dataf_6h[target]


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import pickle

def train(x,y):
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.8)

  # Initialize the model
  model = RandomForestRegressor(max_features=None,n_estimators=250)

  # Train the model

  model.fit(X_train, y_train)

  # Predict on the test set

  y_pred = model.predict(X_test)

  return model

filename = 'model11.sav'

with open(filename, 'wb') as f:
    pickle.dump(model, f)


In [ ]:
model1=train(x_3m,y_5m)
model2=train(x_5m,y_5m)
model3=train(x_15m,y_15m)
model4=train(x_30m,y_30m)
model5=train(x_1h,y_1h)
model6=train(x_2h,y_2h)
model7=train(x_4h,y_4h)
model8=train(x_6h,y_6h)

In [97]:
df_3m.head()

,datetime,open,high,low,close,volume
0,2018-01-01 05:30:00,13715.65,13715.65,13666.11,13680.00,6.303815
1,2018-01-01 05:33:00,13679.98,13679.98,13554.44,13568.00,33.709206
2,2018-01-01 05:36:00,13568.00,13575.00,13501.01,13575.00,29.645987
3,2018-01-01 05:39:00,13539.99,13575.00,13400.01,13515.26,30.696039
4,2018-01-01 05:42:00,13515.26,13568.66,13473.47,13556.15,23.260966


In [123]:
from datetime import datetime,timedelta

date = df_5m.iloc[1]['datetime']

date_object = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')

# Get the minutes from the datetime object
minutes = date_object.minute
hours=date_object.hour


a=int(minutes/3)
b=int(minutes/15)
c=int(minutes/30)


aa=a*3+3
bb=b*15+15
cc=c*30+30

a1=aa-minutes
b1=bb-minutes
c1=cc-minutes



if minutes%3==0:
  date_3m=date
else:
  date_3m=(date_object+timedelta(minutes=a1)).strftime('%Y-%m-%d %H:%M:%S')

if minutes%15==0:
  date_15m=date
else:
  date_15m=(date_object+timedelta(minutes=b1)).strftime('%Y-%m-%d %H:%M:%S')

if minutes%30==0:
  date_30m=date
else:
  date_30m=(date_object+timedelta(minutes=c1)).strftime('%Y-%m-%d %H:%M:%S')

if minutes==30:
  date_1h=date
else:
  date_1h=(date_object+timedelta(hours=1)).replace(minute=30, second=0).strftime('%Y-%m-%d %H:%M:%S')

if minutes==30 and hours%2==1:
  date_2h=date
else:
  date_2h=(date_object+timedelta(hours=2)).replace(minute=30, second=0).strftime('%Y-%m-%d %H:%M:%S')
'''
if minutes==30 and hours%4==1:
  date_4h=date
else:
  date_4h=(date_object+timedelta(hours=4)).strftime('%Y-%m-%d %H:%M:%S')

if minutes==30 and hours%6==1:
  date_6h=date
else:
  date_6h=(date_object+timedelta(hours=6)).strftime('%Y-%m-%d %H:%M:%S')'''


print(date_3m)
print(date_15m)
print(date_30m)
print(date_1h)
print(date_2h)
#print(date_4h)
#print(date_6h)


2018-01-01 05:36:00
2018-01-01 05:45:00
2018-01-01 06:00:00
2018-01-01 06:30:00
2018-01-01 07:30:00


In [127]:
df_4h

,datetime,open,high,low,close,volume
0,2018-01-01 05:30:00,13715.65,13715.65,13155.38,13410.03,1676.204807
1,2018-01-01 09:30:00,13434.98,13818.55,13322.15,13570.35,1302.214836
2,2018-01-01 13:30:00,13569.98,13735.24,13001.13,13220.56,1319.755931
3,2018-01-01 17:30:00,13220.56,13330.00,12750.00,13247.00,1831.933153
4,2018-01-01 21:30:00,13247.00,13290.65,12940.00,13240.37,1092.337234
...,...,...,...,...,...,...
8926,2022-01-30 13:30:00,38244.98,38359.26,37795.00,38110.02,4712.873740
8927,2022-01-30 17:30:00,38110.03,38153.82,37712.05,37980.09,2934.129800
8928,2022-01-30 21:30:00,37977.47,38074.76,37442.52,37512.37,3351.248670
8929,2022-01-31 01:30:00,37512.37,38230.00,37351.63,37881.76,4375.453430


In [ ]:
balance_usdt = 10000
balance_btc = 0
stop_loss = balance_btc*(0.002)

for i in range(df.shape[0]):

    # get the date for the i th row
    date = df.iloc[i]['datetime']

    date_object = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')

    # Get the minutes from the datetime object
    minutes = date_object.minute
    hours=date_object.hour

    a=int(minutes/3)
    b=int(minutes/15)
    c=int(minutes/30)

    aa=a*3+3
    bb=b*15+15
    cc=c*30+30

    a1=aa-minutes
    b1=bb-minutes
    c1=cc-minutes

    if minutes%3==0:
      date_3m=date
    else:
      date_3m=(date_object+timedelta(minutes=a1)).strftime('%Y-%m-%d %H:%M:%S')

    if minutes%15==0:
      date_15m=date
    else:
      date_15m=(date_object+timedelta(minutes=b1)).strftime('%Y-%m-%d %H:%M:%S')

    if minutes%30==0:
      date_30m=date
    else:
      date_30m=(date_object+timedelta(minutes=c1)).strftime('%Y-%m-%d %H:%M:%S')

    if minutes==30:
      date_1h=date
    else:
      date_1h=(date_object+timedelta(hours=1)).replace(minute=30, second=0).strftime('%Y-%m-%d %H:%M:%S')

    if minutes==30 and hours%2==1:
      date_2h=date
    else:
      date_2h=(date_object+timedelta(hours=2)).replace(minute=30, second=0).strftime('%Y-%m-%d %H:%M:%S')
        # Get the data for the date

    data_3m = df[df['datetime'] == date_3m]
    data_5m = df[df['datetime'] == date]
    data_15m = df[df['datetime'] == date_15m]
    data_30m = df[df['datetime'] == date_30m]
    data_1h = df[df['datetime'] == date_1h]
    data_2h = df[df['datetime'] == date_2h]

    # Get the feature vector for the date
    xp_3m = data_3m[features]
    xp_5m = data_5m[features]
    xp_15m = data_15m[features]
    xp_30m = data_30m[features]
    xp_1h = data_1h[features]
    xp_2h = data_2h[features]



    # Predict the price using the model
    y_pred_3m = model1.predict(xp_3m)
    y_pred_5m = model2.predict(xp_5m)
    y_pred_15m = model3.predict(xp_15m)
    y_pred_30m = model4.predict(xp_30m)
    y_pred_1h = model5.predict(xp_1h)
    y_pred_2h = model6.predict(xp_2h)
    #y_pred_4h = model_4h.predict(x)
    #y_pred_6h = model_6h.predict(x)

    # difference between the predicted close and actual open
    open=data_5m["open"].values[0]

    close_predicted=y_pred_3m[0]
    diff1 = close_predicted-open

    close_predicted=y_pred_5m[0]
    diff2 = close_predicted-open

    close_predicted=y_pred_15m[0]
    diff3 = close_predicted-open

    close_predicted=y_pred_30m[0]
    diff4 = close_predicted-open

    close_predicted=y_pred_1h[0]
    diff5 = close_predicted-open

    close_predicted=y_pred_2h[0]
    diff6 = close_predicted-open
    '''
    close_predicted=y_pred_4h[0]
    diff7 = close_predicted-open

    close_predicted=y_pred_6h[0]
    diff8 = close_predicted-open'''

    sum1=0

    if diff1>0:
        sum1=sum1+1
    if diff2>0:
        sum1=sum1+1
    if diff3>0:
        sum1=sum1+1
    if diff4>0:
        sum1=sum1+1

    sum2=0

    if diff5>0:
        sum2=sum2+1
    if diff6>0:
        sum2=sum2+1
    '''if diff7>0:
        sum2=sum2+1
    if diff8>0:
        sum2=sum2+1'''

    # Compare the predicted class with the actual class
    if sum1==4 and balance_usdt>0 and current_row['MACD'] > current_row['Signal_Line']:
        btc=(balance_usdt-0.0015*balance_usdt)/open
        stop_loss = balance_usdt*(0.002)
        balance_btc=btc
        balance_usdt=0
        print(f'{date} -> : usdt : {balance_usdt}   btc : {balance_btc}   [buying]')

    elif sum1==3 and balance_usdt>0:
        btc=((balance_usdt*(0.75))-0.0015*balance_usdt)/open
        stop_loss = balance_usdt*0.75*(0.002)
        balance_btc=btc
        balance_usdt=balance_usdt*(0.25)
        print(f'{date} -> : usdt : {balance_usdt}   btc : {balance_btc}   [buying]')

    elif sum1==2 and sum2>=2 and balance_usdt>0:
        btc=((balance_usdt*(0.50))-0.0015*balance_usdt)/open
        stop_loss = balance_usdt*0.50*(0.002)
        balance_btc=btc
        balance_usdt=balance_usdt*(0.50)
        print(f'{date} -> : usdt : {balance_usdt}   btc : {balance_btc}   [buying]')

    elif sum1<=1 and diff < (-stop_loss) and balance_btc>0:  # Changed this line
        usdt=balance_btc*open
        balance_usdt=usdt-(0.0015*usdt)
        balance_btc=0
        print(f'{date} -> : usdt : {balance_usdt}   btc : {balance_btc}   [selling]')
    else:
        print(f'{date} -> : usdt : {balance_usdt}   btc : {balance_btc}   [holding]')

    #if loop reached the end convert all btc to usd
    if i==df.shape[0]-1 and balance_usdt==0:
        usdt=balance_btc*open
        balance_usdt=usdt
        balance_btc=0
    #print the balance and btc

# Print the final balances after executing the trades
print(f"Final Balances: USDT {balance_usdt}, BTC {balance_btc}")

In [ ]:
correct=0
wrong=0
for i in range(df.shape[0]):

    # Get the date for the i-th row
    date = df.iloc[i]['datetime']

    # Get the data for the date
    data = df[df['datetime'] == date]

    # Get the feature vector for the date
    x = data[features]

    # Predict the price using the model
    y_pred = model.predict(x)

    # Difference between the predicted close and actual open
    open_price = data["open"].values[0]
    close_price = data["close"].values[0]
    diff_actual=close_price-open_price
    close_predicted = y_pred[0]
    diff_predicted=close_predicted-open_price

    if (diff_predicted>0 and diff_actual>0) or (diff_predicted<0 and diff_actual<0):
        correct+=1
    else:
        wrong+=1
    print(f'correct predictions : {correct}     wrong predictions : {wrong}')

NameError: name 'df' is not defined

In [ ]:
# Set the date range
start_date = '2018-06-01'
end_date = '2018-07-31'

# Filter the data
filtered_df1 = df_3m[(df_3m['datetime'] >= start_date) & (df_3m['datetime'] <= end_date)]
print(filtered_df1)

                   datetime     open     high      low    close      volume  \
71644   2018-06-01 00:00:00  7567.91  7579.77  7563.00  7579.50   35.375311   
71645   2018-06-01 00:03:00  7573.83  7579.20  7565.00  7569.18   31.364011   
71646   2018-06-01 00:06:00  7569.11  7576.00  7565.00  7573.84   26.330091   
71647   2018-06-01 00:09:00  7573.84  7575.25  7563.19  7564.00   40.101612   
71648   2018-06-01 00:12:00  7564.00  7569.00  7555.00  7560.06   58.762706   
...                     ...      ...      ...      ...      ...         ...   
100052  2018-07-30 23:45:00  7928.00  7944.00  7920.00  7925.90  183.992499   
100053  2018-07-30 23:48:00  7925.90  7929.96  7901.27  7919.12  167.959768   
100054  2018-07-30 23:51:00  7919.11  7925.90  7901.00  7917.00  176.242756   
100055  2018-07-30 23:54:00  7918.00  7936.30  7901.00  7922.43  142.238363   
100056  2018-07-30 23:57:00  7922.43  7934.79  7920.21  7927.00   64.627551   

              rsi  
71644   42.310750  
71645   41.

In [ ]:
#remove the nan values and replace them to 0

filtered_df1 = filtered_df1.fillna(0)

In [ ]:
import pandas as pd

# Assuming your original dataset is in a CSV file named 'your_dataset.csv'
# Load the dataset into a pandas DataFrame

# Define the number of days to consider (e.g., 10 days)
num_days = 10

# Create new columns for each day
for i in range(1, num_days + 1):
    filtered_df1[f'openday{i}'] = filtered_df1['open'].shift(i)
    filtered_df1[f'highday{i}'] = filtered_df1['high'].shift(i)
    filtered_df1[f'lowday{i}'] = filtered_df1['low'].shift(i)
    filtered_df1[f'closeday{i}'] = filtered_df1['close'].shift(i)
    filtered_df1[f'volumeday{i}'] = filtered_df1['volume'].shift(i)
    filtered_df1[f'rsiday{i}'] = filtered_df1['rsi'].shift(i)

# Drop rows with NaN values created by shifting
df1 = filtered_df1.dropna()

#drop the high low volume and rsi columns

df1 = df1.drop(['high', 'low', 'volume', 'rsi'], axis=1)


In [ ]:
df1=df1.iloc[10:]

In [ ]:
correct=0
wrong=0
for i in range(df1.shape[0]):

    # Get the date for the i-th row
    date = df1.iloc[i]['datetime']

    # Get the data for the date
    data = df1[df1['datetime'] == date]

    # Get the feature vector for the date
    x = data[features]

    # Predict the price using the model
    y_pred = model.predict(x)

    # Difference between the predicted close and actual open
    open_price = data["open"].values[0]
    close_price = data["close"].values[0]
    diff_actual=close_price-open_price
    close_predicted = y_pred[0]
    diff_predicted=close_predicted-open_price

    if (diff_predicted>0 and diff_actual>0) or (diff_predicted<0 and diff_actual<0):
        correct+=1
    else:
        wrong+=1
    print(f'correct predictions : {correct}     wrong predictions : {wrong}')

In [2]:
import tensorflow as tf

# Load the model
model = tf.keras.models.load_model('models/BTC_LSTM_cp_1704897479.313657.h5')

In [12]:
df.dropna(inplace=True)

In [21]:
df.reset_index(inplace=True)

In [36]:
predictions=[]
for i in range(15, len(df)):
    predictions.append(model.predict(df['close'][i-14:i]))


1/1 [==============================] - 0s 17ms/step


KeyboardInterrupt: 

In [23]:
features = ["open",	"high",	"low","volume",	"SMA_14",	"SMA_20",	"RSI",	"MACD",	"Signal_Line",	"Upper_Band",	"Lower_Band",	"EMA",	"ATR",	"short_ma",	"long_ma"]

In [34]:
balance_usdt = 10000
balance_btc = 0
stop_loss = balance_btc*(0.002)

for i in range(14, df.shape[0]):

    # get the date for the i th row
    date = df.iloc[i]['datetime']

    # Get the data for the date
    data = df[df['datetime'] == date]

    # Get the feature vector for the date
    x = data[features]


    # Predict the price using the model
    y_pred = model.predict(df['close'][i-14:i])[0]
    print(y_pred)

    # difference between the predicted close and actual open
    open=data["open"].values[0]
    close_predicted=y_pred[0]
    diff = close_predicted-open

    # Compare the predicted class with the actual class
    if diff > 0 and balance_usdt>0:
        btc=(balance_usdt-0.0015*balance_usdt)/open
        stop_loss = balance_usdt*(0.002)
        balance_btc=btc
        balance_usdt=0
        print(f'{date} -> : usdt : {balance_usdt}   btc : {balance_btc}   [buying]')

    elif diff<0 and diff < (-stop_loss) and balance_btc>0:  # Changed this line
        usdt=balance_btc*open
        balance_usdt=usdt-(0.0015*usdt)
        balance_btc=0
        print(f'{date} -> : usdt : {balance_usdt}   btc : {balance_btc}   [selling]')
    else:
        print(f'{date} -> : usdt : {balance_usdt}   btc : {balance_btc}   [holding]')

    #if loop reached the end convert all btc to usd
    if i==df.shape[0]-1 and balance_usdt==0:
        usdt=balance_btc*open
        balance_usdt=usdt
        balance_btc=0
    #print the balance and btc

# Print the final balances after executing the trades
print(f"Final Balances: USDT {balance_usdt}, BTC {balance_btc}")

1/1 [==============================] - 0s 25ms/step
[[2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]]
2018-01-01 07:09:00 -> : usdt : 10000   btc : 0   [holding]
1/1 [==============================] - 0s 20ms/step
[[2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]]
2018-01-01 07:12:00 -> : usdt : 10000   btc : 0   [holding]
1/1 [==============================] - 0s 21ms/step
[[2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]]
2018-01-01 07:15:00 -> : usdt : 10000   btc : 0   [holding]
1/1 [==============================] - 0s 19ms/step
[[2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.788183]
 [2.78818

KeyboardInterrupt: 

In [ ]:
balance_usdt = 10000
balance_btc = 0
stop_loss = balance_btc*(0.002)

for i in range(df.shape[0]):

    # get the date for the i th row
    date = df.iloc[i]['datetime']

    date_object = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')

    # Get the minutes from the datetime object
    minutes = date_object.minute
    hours=date_object.hour

    a=int(minutes/3)
    b=int(minutes/15)
    c=int(minutes/30)

    aa=a*3+3
    bb=b*15+15
    cc=c*30+30

    a1=aa-minutes
    b1=bb-minutes
    c1=cc-minutes

    if minutes%3==0:
      date_3m=date
    else:
      date_3m=(date_object+timedelta(minutes=a1)).strftime('%Y-%m-%d %H:%M:%S')

    if minutes%15==0:
      date_15m=date
    else:
      date_15m=(date_object+timedelta(minutes=b1)).strftime('%Y-%m-%d %H:%M:%S')

    if minutes%30==0:
      date_30m=date
    else:
      date_30m=(date_object+timedelta(minutes=c1)).strftime('%Y-%m-%d %H:%M:%S')

    if minutes==30:
      date_1h=date
    else:
      date_1h=(date_object+timedelta(hours=1)).replace(minute=30, second=0).strftime('%Y-%m-%d %H:%M:%S')

    if minutes==30 and hours%2==1:
      date_2h=date
    else:
      date_2h=(date_object+timedelta(hours=2)).replace(minute=30, second=0).strftime('%Y-%m-%d %H:%M:%S')
        # Get the data for the date 

    data_3m = df[df['datetime'] == date_3m]
    data_5m = df[df['datetime'] == date]
    data_15m = df[df['datetime'] == date_15m]
    data_30m = df[df['datetime'] == date_30m]
    data_1h = df[df['datetime'] == date_1h]
    data_2h = df[df['datetime'] == date_2h]

    # Get the feature vector for the date
    xp_3m = data_3m[features]
    xp_5m = data_5m[features]
    xp_15m = data_15m[features]
    xp_30m = data_30m[features]
    xp_1h = data_1h[features]
    xp_2h = data_2h[features]



    # Predict the price using the model
    y_pred_3m = model1.predict(xp_3m)
    y_pred_5m = model2.predict(xp_5m)
    y_pred_15m = model3.predict(xp_15m)
    y_pred_30m = model4.predict(xp_30m)
    y_pred_1h = model5.predict(xp_1h)
    y_pred_2h = model6.predict(xp_2h)
    #y_pred_4h = model_4h.predict(x)
    #y_pred_6h = model_6h.predict(x)

    # difference between the predicted close and actual open
    open=data_5m["open"].values[0]

    close_predicted=y_pred_3m[0]
    diff1 = close_predicted-open

    close_predicted=y_pred_5m[0]
    diff2 = close_predicted-open

    close_predicted=y_pred_15m[0]
    diff3 = close_predicted-open

    close_predicted=y_pred_30m[0]
    diff4 = close_predicted-open

    close_predicted=y_pred_1h[0]
    diff5 = close_predicted-open

    close_predicted=y_pred_2h[0]
    diff6 = close_predicted-open
    '''
    close_predicted=y_pred_4h[0]
    diff7 = close_predicted-open

    close_predicted=y_pred_6h[0]
    diff8 = close_predicted-open'''

    sum1=0

    if diff1>0:
        sum1=sum1+1
    if diff2>0:
        sum1=sum1+1
    if diff3>0:
        sum1=sum1+1
    if diff4>0:
        sum1=sum1+1

    sum2=0

    if diff5>0:
        sum2=sum2+1
    if diff6>0:
        sum2=sum2+1
    '''if diff7>0:
        sum2=sum2+1
    if diff8>0:
        sum2=sum2+1'''

    #  100% investment
    if sum1==4 and currentrow['rsi']<30 and currentrow['short_ma']>currentrow['long_ma'] and  balance_usdt>0 and current_row['MACD'] > current_row['Signal_Line'] :
        btc=btc+((balance_usdt-0.0015*balance_usdt)/open)
        stop_loss = balance_usdt*(0.002)
        balance_btc=btc
        balance_usdt=0
        print(f'{date} -> : usdt : {balance_usdt}   btc : {balance_btc}   [buying]')

    #  75% investment
    elif sum1==3 and currentrow['rsi']<30 and currentrow['short_ma']>currentrow['long_ma'] and balance_usdt>0 and current_row['MACD'] > current_row['Signal_Line'] :
        btc=btc + (((balance_usdt*(0.75))-0.0015*balance_usdt)/open)
        stop_loss = balance_usdt*0.75*(0.002)
        balance_btc=btc
        balance_usdt=balance_usdt*(0.25)
        print(f'{date} -> : usdt : {balance_usdt}   btc : {balance_btc}   [buying]')

    #  50% investment
    elif sum1==2 and currentrow['rsi']<30 and currentrow['short_ma'] > currentrow['long_ma'] and sum2>=2 :
        btc=btc+(((balance_usdt*(0.50))-0.0015*balance_usdt)/open)
        stop_loss = balance_usdt*0.50*(0.002)
        balance_btc=btc
        balance_usdt=balance_usdt*(0.50)
        print(f'{date} -> : usdt : {balance_usdt}   btc : {balance_btc}   [buying]')

    elif sum1>=2 and balance_usdt<0:
        
        print(f'{date} -> : usdt : {balance_usdt}   btc : {balance_btc}   [holding]') 

    #25% sell
    elif sum1>=0 and rsi>70 and currentrow['short_ma'] < currentrow['long_ma'] and diff < (-stop_loss) and balance_btc>0 :  # Changed this line
        usdt=balance_usdt+(balance_btc*0.25*open)
        balance_usdt=usdt-(0.0015*usdt)
        balance_btc=balance_btc*0.75
        print(f'{date} -> : usdt : {balance_usdt}   btc : {balance_btc}   [selling]')
  
    #50% sell
    elif sum1>=-2 and sum2>=-1 and rsi>70 and currentrow['short_ma'] < currentrow['long_ma'] and diff < (-stop_loss) and balance_btc>0 and current_row['MACD'] < current_row['Signal_Line'] :   # Changed this line
        usdt=balance_usdt+(balance_btc*0.5*open)
        balance_usdt=usdt-(0.0015*usdt)
        balance_btc=balance_btc*0.50
        print(f'{date} -> : usdt : {balance_usdt}   btc : {balance_btc}   [selling]')

    #75% sell
    elif sum1>=-3 and sum2>=-2 and rsi>70 and currentrow['short_ma'] < currentrow['long_ma'] and diff < (-stop_loss) and balance_btc>0 and current_row['MACD'] < current_row['Signal_Line'] :  # Changed this line
        usdt=balance_usdt+(balance_btc*0.75*open)
        balance_usdt=usdt-(0.0015*usdt)
        balance_btc=balance_btc*0.25
        print(f'{date} -> : usdt : {balance_usdt}   btc : {balance_btc}   [selling]')

    # 100% sell
    elif sum1==-4 and rsi>70 and currentrow['short_ma'] < currentrow['long_ma'] and diff < (-stop_loss) and balance_btc>0 and current_row['MACD'] < current_row['Signal_Line'] :  # Changed this line
        usdt=balance_btc*open
        balance_usdt=usdt-(0.0015*usdt)
        balance_btc=0
        print(f'{date} -> : usdt : {balance_usdt}   btc : {balance_btc}   [selling]')

    else:
        print(f'{date} -> : usdt : {balance_usdt}   btc : {balance_btc}   [holding]')

    #if loop reached the end convert all btc to usd
    if i==df.shape[0]-1 and balance_usdt==0:
        usdt=balance_btc*open
        balance_usdt=usdt
        balance_btc=0
    #print the balance and btc

# Print the final balances after executing the trades
print(f"Final Balances: USDT {balance_usdt}, BTC {balance_btc}")